Let's wrap Google's `snappy` compression library for fun and profit. Let's inspect the library to see what functions are exposed
``` 
> nm /usr/local/lib/libsnappy.dylib

(lots of mangled functions ...)

0000000000003476 T _snappy_compress
00000000000034ca T _snappy_max_compressed_length
00000000000034d4 T _snappy_uncompress
0000000000003540 T _snappy_uncompressed_length
0000000000003551 T _snappy_validate_compressed_buffer
```

Let's now look at `/usr/local/include/snappy-c.h` to get the prototype for these functions.

``` c

snappy_status snappy_compress(const char* input,
                              size_t input_length,
                              char* compressed,
                              size_t* compressed_length);

snappy_status snappy_uncompress(const char* compressed,
                                size_t compressed_length,
                                char* uncompressed,
                                size_t* uncompressed_length);
                                
size_t snappy_max_compressed_length(size_t source_length);

snappy_status snappy_uncompressed_length(const char* compressed,
                                          size_t compressed_length,
                                          size_t* result);

snappy_status snappy_validate_compressed_buffer(const char* compressed, size_t compressed_length);
```
and we also see that `snappy_status` is just a enum with 3 possible values: 0, 1, 2. According to the documentation, the size of the output buffer should be determined using `snappy_max_compressed_length(size_t input_length)`. Similarly, we can get a feel for how the `snappy_uncompress` function works. Let's now wrap them in Python!

In [9]:
from ctypes import cdll, create_string_buffer, byref, c_int

The first step is to import the C functions. This is pretty simple

In [110]:
lsn = cdll.LoadLibrary("libsnappy.dylib")  # This looks for the library using the system's path for dynamic libraries
_snappy_compress = lsn.snappy_compress
_snappy_uncompress = lsn.snappy_uncompress
_snappy_uncompressed_length = lsn.snappy_uncompressed_length
_snappy_validate_compressed_buffer = lsn.snappy_validate_compressed_buffer
snappy_max_compressed_length = lsn.snappy_max_compressed_length # This function doesn't require any wrapping
SNAPPY_OK = 0

Now the interesting part. We should write a convenient wrapper for the C functions. For `snappy_max_compressed_length` we don't even need to do that, as it only takes an int as a parameter, so ctypes can do the required conversion automatically for us. More interestingly, other functions use buffers to store their outputs. We can use `ctypes.create_string_buffer` to create the actual buffers, and `ctypes.byref` to pass a reference to a C type (e.g. `ctypes.c_int`).

In [109]:
def snappy_compress(input_data):
    input_l = len(input_data)
    buf_size = snappy_max_compressed_length(input_l)
    o_s = c_int(buf_size)
    output_buf = create_string_buffer("\000" * buf_size)
    sn_code = _snappy_compress(input_data, input_l, output_buf, byref(o_s))
    if sn_code != SNAPPY_OK:
        raise Exception("Snappy failed with code %i" % sn_code)
    # After the call to the C function, `output_buf` contains the
    # compressed data and `o_s` the compressed length.
    # Let's return only the used portion of the output buffer. 
    return output_buf[:o_s.value]
                                      
def snappy_uncompressed_length(compressed):
    out = c_int()
    _snappy_uncompressed_length(compressed, len(compressed), byref(out))
    return out.value

def snappy_uncompress(compressed):
    compressed_l = len(compressed)
    buf_size = snappy_uncompressed_length(compressed)
    out_buf = create_string_buffer("\000" * buf_size)
    out_l = c_int(buf_size)
    sn_code = _snappy_uncompress(compressed, compressed_l, out_buf, byref(out_l))
    if sn_code != SNAPPY_OK:
        raise Exception("Snappy failed with code %i" % sn_code)
    return out_buf[:out_l.value]
    
def snappy_validate_compressed_buffer(compressed):
    out = _snappy_validate_compressed_buffer(compressed, len(compressed))
    return out

Let's now use this functions to compress and decompress a file

In [105]:
s = open("/Users/dario/.vimrc").read()
print "Original length: %i" % len(s)
compressed = snappy_compress(s)
print "Compressed length: %i" % len(compressed)

Original length: 6851
Compressed length: 3544


In [106]:
rec = snappy_uncompress(compressed)

In [107]:
print len(rec)

6851


In [108]:
print rec

syntax on
filetype indent plugin on
" set background=light
let backgroundEnv=$BACKGROUND
if backgroundEnv == 'light'
    set background=light
else
    set background=dark
endif

" Add support for Pig syntax
augroup filetypedetect
 au BufNewFile,BufRead *.pig set filetype=pig syntax=pig
augroup END

" Interpret Hive files as SQL
au BufNewFile,BufRead *.hql set filetype=sql

" Markdown
" Don't fold
set nofoldenable

execute pathogen#infect()

let mapleader = ","
let g:mapleader = ","

set so=7
set number

set wildmenu
set wildignore=*.o,*.pyc,*~

set ruler

set cmdheight=2

set backspace=eol,start,indent
set whichwrap+=<,>,h,l

set hlsearch
set incsearch

set lazyredraw

set magic
set noerrorbells
set t_vb=
set tm=500

set expandtab
set smarttab
set tabstop=4
set shiftwidth=4

set autochdir

" Slime
let g:slime_target = "tmux"

" Tell ctrl+p to use git ls-files on git repos
" let g:ctrlp_user_command = ['.git/', 'git --git-dir=%s/.git ls-files -oc --exclude-standard']
" The Silver Search

In [103]:
snappy_validate_compressed_buffer(compressed)

0